# O1v5-SONATA connectome manipulation preparation to add reciprocal connections
This is an auxiliary notebook for configuring model building and rewiring, which...
 - creates connectivity models from adjacency matrices containing additional reciprocal connections
 - sets up rewiring using these connectivity models to add reciprocal connections to the original connectome

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.10.dev1 or later)

ℹ️ Related ticket: [[ACCS-62](https://bbpteam.epfl.ch/project/issues/browse/ACCS-62)] "Adding reciprocal connections to V5 connectome"


In [1]:
# Initialization
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tqdm
from bluepysnap import Circuit
from connectome_manipulator.model_building import conn_prob_adj
from scipy import sparse as sps

__Load and check adjacency matrix with connections to be added__

In [47]:
# Specify adjacency matrices for adding connections
adj_seed = 0  # Random seed
adj_factor = 4  # Increase factor of RC's per dimension
adj_path = "/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data"

## Structured
adj_file = os.path.join(adj_path, "mats_rc_on_simplices.pkl")
adj_name = f"struct{adj_seed}_{adj_factor}x"
check_rc = True  # Check that added connections are reciprocal

## Control
# adj_file = os.path.join(adj_path, "mats_rc_on_simplices_controls.pkl")
# adj_name = f"control{adj_seed}_{adj_factor}x"
# check_rc = False  # Don't check that added connections are reciprocal

# Specify original adjacency (for checks)
orig_file = os.path.join(adj_path, "mats_rc_on_simplices.pkl")
orig_key = "original"

In [48]:
# Load original & structured/control adjacency matrices
with open(orig_file, "rb") as f:
    tmp_dict = pickle.load(f)
orig_adj = tmp_dict[orig_key]
print(f'Loaded "{orig_key}" adjacency matrix with {orig_adj.count_nonzero()} edges')

with open(adj_file, "rb") as f:
    tmp_dict = pickle.load(f)
adj = tmp_dict[f"modified_{adj_factor}"][adj_seed]
print(f'Loaded "{adj_name}" adjacency matrix with {adj.count_nonzero()} edges')

Loaded "original" adjacency matrix with 6717001 edges
Loaded "struct0_4x" adjacency matrix with 7384380 edges


In [49]:
# Load circuit & node ids corresponding to adjacency matrix
circuit_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA'
circuit_name = os.path.split(circuit_path)[-1]
circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config_tmp.json') # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]

c = Circuit(circuit_config)
nodes = c.nodes["default"]
edges = c.edges["default"]

mc2_nodeset = "mc2_Column"
syn_sel = "EXC"
nids = np.intersect1d(nodes.ids(mc2_nodeset), nodes.ids({"synapse_class": syn_sel}))

assert adj.shape[0] == adj.shape[1] == len(nids), "ERROR: Nodes inconsistent with size of adjacency matrix!"
assert orig_adj.shape[0] == orig_adj.shape[1] == len(nids), "ERROR: Nodes inconsistent with size of original adjacency matrix!"

In [50]:
# Consistency check with original O1v5 adjacency matrices

## From structural comparator
with open("/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/struct_comparison/O1v5-SONATA/data/Orig_Adjacency_mc2EE.pickle", "rb") as f:
    tmp_dict = pickle.load(f)
tmp_adj = tmp_dict["adj"]["data"]
assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (structural comparator)!"

## From baseline sims
tmp_adj = sps.load_npz("/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline/working_dir/connectivity.npz")
tmp_sel = np.isin(nodes.ids(mc2_nodeset), nids)
tmp_adj = tmp_adj[:, tmp_sel][tmp_sel, :]
assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (baseline sims)!"

## From toposample paper (zenodo)
# tmp_adj = sps.load_npz("/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/PLOS_ONE_Toposample_Paper/zenodo/input_data/connectivity.npz")
# tmp_sel = np.isin(nodes.ids(mc2_nodeset), nids)
# tmp_adj = tmp_adj[:, tmp_sel][tmp_sel, :]
# assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (toposample)!"
# IGNORE: Adjacency has 6717002 edges for some reason; one additional edge is at index tmp_adj[26566, 1141]

## Also directly checked with bluepy (old v5 version), bluepysnap (SONATA version), and conntility (SONATA version): 6717001 edges

In [51]:
# Check edges to be added
assert np.all(adj[orig_adj.nonzero()]), "ERROR: Connections inconsistent with original adjacency matrix!"
diff_mat = adj - orig_adj
num_add = diff_mat.size
print(f'Connections to be added in "{adj_name}": {num_add}')

# Check reciprocity
if check_rc:
    assert not np.any(orig_adj[diff_mat.nonzero()]), "ERROR: Edges to be added already exist!"
    assert np.all(orig_adj.T[diff_mat.nonzero()]), "ERROR: Edges to be added are not reciprocal!"
    print("Checked reciprocity ... OK")

Connections to be added in "struct0_4x": 667379
Checked reciprocity ... OK


__Create and save adjacency model for rewiring__

In [52]:
adj_model_path = f"/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/model_building/{circuit_name}/model"
if not os.path.exists(adj_model_path):
    os.makedirs(adj_model_path)

In [53]:
adj_model = conn_prob_adj.build(adj.tocsc(), nids, nids)
adj_model_name = f"ConnProbAdjModel_mc2EE_{adj_name}"
adj_model.save_model(adj_model_path, adj_model_name)
print(adj_model)
adj_model_file = os.path.join(adj_model_path, adj_model_name + ".json")
assert os.path.exists(adj_model_file), f'ERROR: Model file "{adj_model_file}" not saved!'
print(f'Model saved to "{adj_model_file}"')

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.bool_'>'
	with 7384380 stored elements in Compressed Sparse Column format>
Model saved to "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/model_building/O1v5-SONATA/model/ConnProbAdjModel_mc2EE_struct0_4x.json"


__Re-extract ConnPropsModel__

- Model format from "simplified connectome models" is outdated
- Using model building config from "simplified connectome models" with slight modifications to re-run model building

Launch command:

`cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs`

`sbatch run_model_building.sh model_config__ConnPropsPerPathwayO1v5mc2EE.json --force-reextract --force-rebuild`

__Create rewiring config & launch rewiring__

In [54]:
output_base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs']  # Write to local and proj9 folder

In [55]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def add_manip_to_config(manip_config, name, prob_model_file, delay_model_file, props_model_file):
    """Adds a 'ConnAddRC' rewiring operation to the (v4) manipulation config dict (in-place)."""
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    manip_config['manip'] = {'name': f'ConnAddRC_mc2EE_{name}',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'sel_dest': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'keep_conns': True,
                                       'reuse_pos': True,
                                       'rewire_mode': 'add_only',
                                       'gen_method': 'randomize',
                                       'amount_pct': 100.0,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel):
    run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


In [56]:
# Model locations (MC2 column models!!)
delay_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model', 'DistDepDelayO1v5mc2EE.json')
props_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/model_building', circuit_name, 'model', 'ConnPropsPerPathwayO1v5mc2EE.json')
prob_model_file = adj_model_file

In [57]:
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, adj_name, prob_model_file, delay_model_file, props_model_file)
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnAddRC_mc2EE_struct0_4x.json written to ../configs
Config file manip_config__ConnAddRC_mc2EE_struct0_4x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_struct0_4x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_struct0_4x" 500


Launch commands:

`cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs`

`sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_struct0_2x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_struct0_2x" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_struct0_4x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_struct0_4x" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_control0_2x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_control0_2x" 500`

`sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_control0_4x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_control0_4x" 500`


__Extract connection counts (from log files)__

In [3]:
# Check connection/synapse counts
N_split = 500
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_struct0_4x/logs/connectome_manipulation_2023-11-17_16h00.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_struct0_2x/logs/connectome_manipulation_2023-11-20_10h38.log"
# main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_control0_4x/logs/connectome_manipulation_2023-11-17_16h51.log"
main_log_file = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_control0_2x/logs/connectome_manipulation_2023-11-20_10h39.log"

log_path, log_fn = os.path.split(main_log_file)  # Main log path/filename
log_date = os.path.splitext(log_fn)[0].split('connectome_manipulation_')[1]

log_folders = [os.path.join(log_path, d) for d in os.listdir(log_path) if os.path.isdir(os.path.join(log_path, d))]
data_log_splits = [f'{log_date}.RewiringIndices_{i + 1}_{N_split}.npz' for i in range(N_split)]
data_log_splits_alt = [None] * len(data_log_splits)  # Alternative names [not used here]

conn_count_orig_all = 0 # Overall input connection count
conn_count_actu_all = 0 # Overall output connection count after rewiring (actual value)
conn_count_orig_sel = 0 # Input connection count within selected rewiring target
conn_count_actu_sel = 0 # Output connection count within selected rewiring target after rewiring (actual value)
split_count = 0
for split_name, alt_name in tqdm.tqdm(zip(data_log_splits, data_log_splits_alt), desc="Collecting log files", total=N_split):
    # Search for log file in all log folders
    folder_name = None
    file_name = None
    for sp_nm in [split_name, alt_name]:
        if sp_nm is None:
            break
        for fld in log_folders:
            tmp_name = [fn for fn in os.listdir(fld) if sp_nm in fn]
            assert len(tmp_name) <= 1, 'ERROR: Multiple files found in current folder!'
            if len(tmp_name) == 1:  # Match
                assert folder_name is None, 'ERROR: Folder not unique!'
                assert file_name is None, 'ERROR: File not unique!'
                folder_name = fld
                file_name = tmp_name[0]
        if folder_name is not None and file_name is not None:
            break
    assert folder_name is not None and file_name is not None, f'ERROR: Log file ..{split_name} {"" if alt_date is None else "(or alternative)"} not found!'

    # Read log file
    file_name = file_name.replace("Indices", "Stats")  # In any case, the data log to load is the ..Stats.. one, if existing
    dlog = os.path.join(folder_name, file_name)
    if not os.path.exists(dlog):
        continue
    split_count += 1
    stats_dict = np.load(dlog)
    conn_count_orig_sel += np.sum(stats_dict['input_conn_count_sel'])
    conn_count_orig_all += np.sum(stats_dict.get('input_conn_count', 0))
    conn_count_actu_sel += np.sum(stats_dict['output_conn_count_sel'])
    assert stats_dict['output_conn_count'] == np.sum(stats_dict['output_conn_count_sel']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
    conn_count_actu_all += stats_dict['output_conn_count']

print(f'Non-empty splits: {split_count}/{N_split}')
print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_rewired = {conn_count_actu_all}, DIFF = {conn_count_actu_all - conn_count_orig_all} ({100.0 * (conn_count_actu_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_rewired = {conn_count_actu_sel}, DIFF = {conn_count_actu_sel - conn_count_orig_sel} ({100.0 * (conn_count_actu_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')

Non-empty splits: 72/500
Overall: #Conns_orig = 14138043, #Conns_rewired = 14366192, DIFF = 228149 (1.61%)
Within wiring target: #Conns_orig = 6717001, #Conns_rewired = 6945150, DIFF = 228149 (3.40%)


Rewiring statistics:

- `mc2EE_struct0_2x`: Added 228149 connections, 751635 synapses
- `mc2EE_struct0_4x`: Added 667379 connections, 2198050 synapses
- `mc2EE_control0_2x`: Added 228149 connections, 686149 synapses
- `mc2EE_control0_4x`: Added 667379 connections, 2008793 synapses

__Check connectomes__

In [2]:
# Load circuit & node ids corresponding to adjacency matrix
c = Circuit("/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA/sonata/circuit_config_tmp.json")
nodes = c.nodes["default"]
mc2_nodeset = "mc2_Column"
nids = np.intersect1d(nodes.ids(mc2_nodeset), nodes.ids({"synapse_class": "EXC"}))

In [13]:
# Adjacency list [<filename>, <blowup>, <seed>]
adj_list = [# ["/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices.pkl", 2, 0],
            ["/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices.pkl", 4, 0],
            # ["/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices_controls.pkl", 2, 0],
            ["/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices_controls.pkl", 4, 0]]
ref_list = [# "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_RecipStruct0x2/working_dir/connectivity.npz",
            "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_RecipStruct0x4/working_dir/connectivity.npz",
            # "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_Control0x2/working_dir/connectivity.npz",
            "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_Control0x4/working_dir/connectivity.npz"]

for adj_file, ref_file in zip(adj_list, ref_list):
    print(f"CHECKING {adj_file}...", end="")
    with open(adj_file[0], "rb") as f:
        tmp_dict = pickle.load(f)
    adj = tmp_dict[f"modified_{adj_file[1]}"][adj_file[2]]

    tmp_adj = sps.load_npz(ref_file)
    tmp_sel = np.isin(nodes.ids(mc2_nodeset), nids)
    ref_adj = tmp_adj[:, tmp_sel][tmp_sel, :]

    assert np.array_equal(adj.toarray(), ref_adj.toarray()), "ERROR: Adjacency mismatch in!"
    print("OK")

CHECKING ['/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices.pkl', 4, 0]...OK
CHECKING ['/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data/mats_rc_on_simplices_controls.pkl', 4, 0]...OK
